# Library:
# python version 3.8.5
# pandas version 0.25.3
# mysql-connector-python version 8.0.33
# ipython-sql version 0.4.1
# sqlalchemy version 2.0.13
# sqlite version 3.33.0
# xlsxwriter version 1.3.7

In [ ]:
!pip install mysql-connector-python
!pip install ipython-sql

In [ ]:
import mysql.connector as connection
import pandas as pd

def sql_query(query):
    try:
        mydb = connection.connect(host="ckhos.go.th", database = 'dhdc',user="your_username", password="your_password",use_pure=True)       
        result_dataFrame = pd.read_sql(query,mydb)
        mydb.close() #close the connection
        return result_dataFrame
    except Exception as e:
        mydb.close()
        print(str(e))

In [ ]:
coc_hdc = pd.read_csv('coc_25660508_all.csv',encoding="utf-8", dtype = str).query('RESULT == "1B1281" or RESULT == "1B1282"')
coc_hdc

In [ ]:
#print(coc_hdc.dtypes)

In [ ]:
coc_hdc['hospid'] = coc_hdc['รหัสสถานบริการHOSCODE'] + coc_hdc['ทะเบียนบุคคลPID']#.astype(str) 
coc_hdc['name'] = coc_hdc['ชื่อNAME'] +' '+ coc_hdc['นามสกุลLNAME']
coc_hdc_left = coc_hdc.loc[:, ['รหัสสถานบริการHOSCODE','ชื่อสถานบริการHOSNAME','name', 'hospid','RESULT']]
coc_hdc_left

In [ ]:
dental_cid = sql_query( """ 
SELECT
DT.hospid,
DT.CID,
DT.flag_dental 

FROM
(
SELECT DISTINCT   
    CONCAT(person.HOSPCODE, person.PID) AS hospid,
    person.CID,
    case when dental.DATE_SERV = max(dental.DATE_SERV) OVER (PARTITION BY dental.CID) 
             then '1' else '0'
    end AS flag_dental 
    
    from dental    
    
    INNER JOIN provider ON provider.provider = dental.provider AND provider.HOSPCODE = dental.HOSPCODE
    INNER JOIN person ON person.cid = dental.cid 
    
    WHERE 
    
    provider.providertype IN ('02','06')
    AND person.TYPEAREA IN ('1', '3')
    AND dental.PTEETH BETWEEN 0 AND 32
    AND DATE_FORMAT(FROM_DAYS(DATEDIFF('2022-10-01',person.BIRTH)), '%Y')+0 BETWEEN 60 AND 199
    AND IFNULL(dental.PERMANENT_PERMANENT,0) + IFNULL(dental.PERMANENT_PROSTHESIS,0)
    + IFNULL(dental.PROSTHESIS_PROSTHESIS,0) BETWEEN 0 AND 10
    AND (IFNULL(dental.pteeth,0) +IFNULL(dental.PEXTRACT,0)) > 0
    AND dental.DATE_SERV BETWEEN '2022-10-01' AND '2023-09-30'
    AND LENGTH(dental.CID)=13 
    
  )  AS DT

where
DT.flag_dental = '1'
""" )

dental_cid

In [ ]:
tb_merge = pd.merge(coc_hdc_left, dental_cid, on=['hospid'], how='left')
tb_merge

In [ ]:
tb_merge.style.hide(axis='index')

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo = False)

In [ ]:
tb_merge.to_sql('COC_Data',
                       con = engine,
                       schema=None,
                       if_exists = 'append')

In [ ]:
coc2 = pd.read_sql('''
SELECT 
Q1.hospcode,
Q1.cup,
Q1.target,
Q1.result,
IFNULL(ROUND(((Q1.result/1.0)/Q1.target)*100, 2), 0.00) AS percent,
Q1.target1,
Q1.result1,
IFNULL(ROUND(((Q1.result1/1.0)/Q1.target1)*100, 2), 0.00) AS percent1,
Q1.target2,
Q1.result2,
IFNULL(ROUND(((Q1.result2/1.0)/Q1.target2)*100, 2), 0.00) AS percent2
 
FROM
(
SELECT DISTINCT
รหัสสถานบริการHOSCODE AS hospcode,
ชื่อสถานบริการHOSNAME AS cup,
COUNT(DISTINCT hospid) AS target,
COUNT(flag_dental) AS result,
COUNT(DISTINCT IIF(RESULT IN('1B1281') ,hospid,NULL)) AS target1,
COUNT(DISTINCT IIF(flag_dental IS NOT NULL AND RESULT IN('1B1281') ,hospid,NULL)) AS result1,
COUNT(DISTINCT IIF(RESULT IN('1B1282') ,hospid,NULL)) AS target2,
COUNT(DISTINCT IIF(flag_dental IS NOT NULL AND RESULT IN('1B1282') ,hospid,NULL)) AS result2

FROM COC_Data

GROUP BY hospcode, cup
) AS Q1
''', engine)

sum_target = coc2.target.sum()
sum_result = coc2.result.sum()
total_percent = ((sum_result/sum_target)*100).round(2)
sum_target1 = coc2.target1.sum()
sum_result1 = coc2.result1.sum()
total_percent1 = ((sum_result1/sum_target1)*100).round(2)
sum_target2 = coc2.target2.sum()
sum_result2 = coc2.result2.sum()
total_percent2 = ((sum_result2/sum_target2)*100).round(2)
coc2.loc[len(coc2.index)] = ['All', 'Total', sum_target, sum_result, total_percent, sum_target1, sum_result1, total_percent1,
                                         sum_target2, sum_result2, total_percent2]
coc2
#coc2.dtypes

In [ ]:
coc2.style.hide(axis='index').format({'percent': '{:.2f}','percent1': '{:.2f}','percent2': '{:.2f}'})

In [ ]:
coc2.to_excel(r'D:\selenium\coc_dental_exam.xlsx', header=True,index=False)